In [40]:
import cv2
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon

import csv
import numpy as np
import tensorflow as tf
import pyautogui as mouse
%matplotlib inline

In [41]:
class HandTracker():
    r"""
    Class to use Google's Mediapipe HandTracking pipeline from Python.
    So far only detection of a single hand is supported.
    Any any image size and aspect ratio supported.
    Args:
        palm_model: path to the palm_detection.tflite
        joint_model: path to the hand_landmark.tflite
        anchors_path: path to the csv containing SSD anchors
    Ourput:
        (21,2) array of hand joints.
    Examples::
        # >>> det = HandTracker(path1, path2, path3)
        # >>> input_img = np.random.randint(0,255, 256*256*3).reshape(256,256,3)
        # >>> keypoints, bbox = det(input_img)
    """

    def __init__(self, palm_model, joint_model, anchors_path,
                 box_enlarge=1.5, box_shift=0.2):
        self.box_shift = box_shift
        self.box_enlarge = box_enlarge

        self.interp_palm = tf.lite.Interpreter(palm_model)
        self.interp_palm.allocate_tensors()
        self.interp_joint = tf.lite.Interpreter(joint_model)
        self.interp_joint.allocate_tensors()

        # reading the SSD anchors
        with open(anchors_path, "r") as csv_f:
            self.anchors = np.r_[
                [x for x in csv.reader(csv_f, quoting=csv.QUOTE_NONNUMERIC)]
            ]
        # reading tflite model paramteres
        output_details = self.interp_palm.get_output_details()
        input_details = self.interp_palm.get_input_details()

        self.in_idx = input_details[0]['index']
        self.out_reg_idx = output_details[0]['index']
        self.out_clf_idx = output_details[1]['index']

        self.in_idx_joint = self.interp_joint.get_input_details()[0]['index']
        self.out_idx_joint = self.interp_joint.get_output_details()[0]['index']

        # 90° rotation matrix used to create the alignment trianlge
        self.R90 = np.r_[[[0, 1], [-1, 0]]]

        # trianlge target coordinates used to move the detected hand
        # into the right position
        self._target_triangle = np.float32([
            [128, 128],
            [128, 0],
            [0, 128]
        ])
        self._target_box = np.float32([
            [0, 0, 1],
            [256, 0, 1],
            [256, 256, 1],
            [0, 256, 1],
        ])

    def _get_triangle(self, kp0, kp2, dist=1):
        """get a triangle used to calculate Affine transformation matrix"""

        dir_v = kp2 - kp0
        dir_v /= np.linalg.norm(dir_v)

        dir_v_r = dir_v @ self.R90.T
        return np.float32([kp2, kp2 + dir_v * dist, kp2 + dir_v_r * dist])

    @staticmethod
    def _triangle_to_bbox(source):
        # plain old vector arithmetics
        bbox = np.c_[
            [source[2] - source[0] + source[1]],
            [source[1] + source[0] - source[2]],
            [3 * source[0] - source[1] - source[2]],
            [source[2] - source[1] + source[0]],
        ].reshape(-1, 2)
        return bbox

    @staticmethod
    def _im_normalize(img):
        return np.ascontiguousarray(
            2 * ((img / 255) - 0.5
                 ).astype('float32'))

    @staticmethod
    def _sigm(x):
        return 1 / (1 + np.exp(-x))

    @staticmethod
    def _pad1(x):
        return np.pad(x, ((0, 0), (0, 1)), constant_values=1, mode='constant')

    def predict_joints(self, img_norm):
        self.interp_joint.set_tensor(
            self.in_idx_joint, img_norm.reshape(1, 256, 256, 3))
        self.interp_joint.invoke()

        joints = self.interp_joint.get_tensor(self.out_idx_joint)
        return joints.reshape(-1, 2)

    def detect_hand(self, img_norm):
        assert -1 <= img_norm.min() and img_norm.max() <= 1, \
            "img_norm should be in range [-1, 1]"
        assert img_norm.shape == (256, 256, 3), \
            "img_norm shape must be (256, 256, 3)"

        # predict hand location and 7 initial landmarks
        self.interp_palm.set_tensor(self.in_idx, img_norm[None])
        self.interp_palm.invoke()

        out_reg = self.interp_palm.get_tensor(self.out_reg_idx)[0]
        out_clf = self.interp_palm.get_tensor(self.out_clf_idx)[0, :, 0]

        # finding the best prediction
        # TODO: replace it with non-max suppression
        detecion_mask = self._sigm(out_clf) > 0.7
        candidate_detect = out_reg[detecion_mask]
        candidate_anchors = self.anchors[detecion_mask]

        if candidate_detect.shape[0] == 0:
            # print("No hands found")
            return None, None
        # picking the widest suggestion while NMS is not implemented
        max_idx = np.argmax(candidate_detect[:, 3])

        # bounding box offsets, width and height
        dx, dy, w, h = candidate_detect[max_idx, :4]
        center_wo_offst = candidate_anchors[max_idx, :2] * 256

        # 7 initial keypoints
        keypoints = center_wo_offst + candidate_detect[max_idx, 4:].reshape(-1, 2)
        side = max(w, h) * self.box_enlarge

        # now we need to move and rotate the detected hand for it to occupy a
        # 256x256 square
        # line from wrist keypoint to middle finger keypoint
        # should point straight up
        # TODO: replace triangle with the bbox directly
        source = self._get_triangle(keypoints[0], keypoints[2], side)
        source -= (keypoints[0] - keypoints[2]) * self.box_shift
        return source, keypoints

    def preprocess_img(self, img):
        # fit the image into a 256x256 square
        shape = np.r_[img.shape]
        pad = (shape.max() - shape[:2]).astype('uint32') // 2
        img_pad = np.pad(
            img,
            ((pad[0], pad[0]), (pad[1], pad[1]), (0, 0)),
            mode='constant')
        img_small = cv2.resize(img_pad, (256, 256))
        img_small = np.ascontiguousarray(img_small)

        img_norm = self._im_normalize(img_small)
        return img_pad, img_norm, pad

    def __call__(self, img):
        img_pad, img_norm, pad = self.preprocess_img(img)

        source, keypoints = self.detect_hand(img_norm)
        if source is None:
            return None, None

        # calculating transformation from img_pad coords
        # to img_landmark coords (cropped hand image)
        scale = max(img.shape) / 256
        Mtr = cv2.getAffineTransform(
            source * scale,
            self._target_triangle
        )

        img_landmark = cv2.warpAffine(
            self._im_normalize(img_pad), Mtr, (256, 256)
        )

        joints = self.predict_joints(img_landmark)

        # adding the [0,0,1] row to make the matrix square
        Mtr = self._pad1(Mtr.T).T
        Mtr[2, :2] = 0

        Minv = np.linalg.inv(Mtr)

        # projecting keypoints back into original image coordinate space
        kp_orig = (self._pad1(joints) @ Minv.T)[:, :2]
        box_orig = (self._target_box @ Minv.T)[:, :2]
        kp_orig -= pad[::-1]
        box_orig -= pad[::-1]

        return kp_orig, box_orig

In [42]:
import os
base = os.path.join(os.path.abspath(""), "tensorflow_test/handTracking/")
palm_model_path = os.path.join(base, "palm_detection_without_custom_op.tflite")
landmark_model_path = os.path.join(base, "hand_landmark.tflite")
anchors_path = os.path.join(base, "anchors.csv")
test_img =  os.path.join(base, "test_img.jpg")

assert os.path.isfile(palm_model_path) 
assert os.path.isfile(landmark_model_path) 
assert os.path.isfile(anchors_path) 
assert os.path.isfile(test_img)
img = cv2.imread(test_img)[:, :, ::-1]
detector = HandTracker(palm_model_path, landmark_model_path, anchors_path,
                   box_shift=0.2, box_enlarge=1.3)

In [4]:
print(img.shape)
ig = cv2.resize(img, (int(img.shape[0]/5), int(img.shape[1]/5)))
print(ig.shape)
cv2.imshow("goog", ig)
cv2.waitKey(0)

(4096, 3072, 3)
(614, 819, 3)


-1

In [26]:
import time
start = time.time()
ig = cv2.resize(img, (1000, 1000))
kp, box = detector(ig)
# print(kp)
# cv2.namedWindow("goog")
# cv2.resizeWindow("goog", 10, 10)
center = []
for i in range(4):
    center.append((kp[5+i*4]+kp[0])/2)
centerPoint = np.sum(center, axis=0)/4
# print(centerPoint)
cv2.circle(ig, (int(centerPoint[0]), int(centerPoint[1])), radius=10, 
               color=(255, 255, 255), thickness=-1,
               lineType=cv2.FILLED)
# for i in center:
#     cv2.circle(ig, (int(i[0]), int(i[1])), radius=4, 
#                color=(0, 255, 255), thickness=-1,
#                lineType=cv2.FILLED)
for i in kp:
    cv2.circle(ig, (int(i[0]), int(i[1])), radius=4, 
               color=(0, 0, 255), thickness=-1,
               lineType=cv2.FILLED)
cv2.imshow("goog", ig)
#print(box)
# f,ax = plt.subplots(1,1, figsize=(10, 10))
# 
# ax.imshow(img)
# 
# ax.scatter(kp[:, 0], kp[:, 1])
# ax.add_patch(Polygon(box, color="#00ff00", fill=False))
end = time.time()
cv2.waitKey(0)
cv2.destroyAllWindows()
print("speed %.6f " % (end-start))

C:\Users\goog\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:89: RuntimeWarning: overflow encountered in exp


speed 0.389896 


In [74]:
cap = cv2.VideoCapture(0)
mouse.FAILSAFE = False
arr_x = np.array([0,0,0], dtype=float)
arr_y = np.array([0,0,0], dtype=float)
avg_x = 0.0
avg_y = 0.0
x = 0.0
y = 0.0

while True:
    
    ret, frame = cap.read()
    # frame = frame[:, ::-1, :]
    frame = cv2.flip(frame, 1)
    
    kp, box = detector(frame)
    # print(frame.shape)
    # break
    if kp is not None:
        # 找到手掌心
        start = time.time()
        center = []
        for i in range(4): 
            center.append((kp[5+i*4]+kp[0])/2)
        centerPoint_x, centerPoint_y = np.sum(center, axis=0)/4
        # 0: 距离左边的距离
        # 1：距离上边的距离
        # img_size = [640, 480]
        # print(centerPoint)
        # 左右上下裁剪100
        # 映射范围裁剪
        
        cv2.circle(frame, (int(centerPoint_x), int(centerPoint_y)), radius=10, 
                       color=(255, 255, 255), thickness=-1,
                       lineType=cv2.FILLED)
        
        # 映射    
        x = 1920*(centerPoint_x-120)/400
        y = 1080*(centerPoint_y-120)/240
        if x<0:
            x = 0
        if y<0:
            y=0
        if x>1920:
            x=1920
        if y>1080:
            y=1080
        arr_y[2] = arr_y[1]
        arr_x[2] = arr_x[1]
        arr_y[1] = arr_y[0]
        arr_x[1] = arr_x[0]
        arr_x[0] = x
        arr_y[0] = y
        
        avg_x = np.mean(arr_x)
        avg_y = np.mean(arr_y)
        mouse.moveTo(int(avg_x), int(avg_y))
        # print(centerPoint)
        
        
        for i in kp:
            cv2.circle(frame, (int(i[0]), int(i[1])), radius=10, 
                       color=(0, 255, 0),
                       thickness=-1, lineType=cv2.FILLED)
    
    cv2.imshow("goog", frame)
    
    end = time.time()
    # print("speed %.6f " % (end-start))
    if cv2.waitKey(20)&0xff == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

C:\Users\goog\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:89: RuntimeWarning: overflow encountered in exp


In [6]:

import time
time.sleep(1)
mouse.dragRel(100, 10, 1)
mouse.doubleClick()